In [47]:
from PIL import Image 
from keras import backend as K
from keras.models import * 
from keras.layers import * 
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler    
import tensorflow as tf
import glob,pickle
import numpy as np 
import tensorflow.gfile as gfile 
import matplotlib.pyplot as mp
import os

In [48]:
CAPTCHA_CHARSET = ['0', '1', '2', '3', '4', '5', '6', '7', 
                   '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 
                   'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                   'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
                   'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd',
                   'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                   'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
                   'u', 'v', 'w', 'x', 'y', 'z']

def text2vec(text, length = 4, charset = CAPTCHA_CHARSET):
    text_len = len(text)
    if text_len != length:
        raise ValueError(
            "输入字符长度为{}，与所需验证码长度{}不相符".format(text_len,length))
    vec = np.zeros(length * len(charset))
    for i in range(length):
        vec[charset.index(text[i]) + i * len(charset)] = 1
    return vec

In [49]:
def vec2text(vector):
    if not isinstance(vector, np.ndarray):
        vector = np.asarray(vector)
    vector = np.reshape(vector, [4, -1])
    text = ''
    for item in vector:
        text += CAPTCHA_CHARSET[np.argmax(item)]
    return text

In [50]:
def fit_keras_channels(batch, rows = 60, cols = 160):
    if K.image_data_format() == 'channel first':
        batch = batch.reshape(batch.shape[0],1,rows,cols)
        input_shape = (1,rows,cols)
    else:
        batch = batch.reshape(batch.shape[0],rows,cols,1)
        input_shape = (rows,cols,1)
    return batch,input_shape

In [51]:
def rgb2gray(image):
    return np.dot(image[...,:3], [0.299,0.587,0.114])

In [52]:
def load_data(path):
    X_train, Y_train = [],[]
    for filename in glob.glob(path + '/' + '*.png'):
        X_train.append(np.array(Image.open(filename)))
        Y_train.append(filename.split('.')[-2].split('\\')[-1])
    return X_train, Y_train

In [53]:
X_train, Y_train = load_data('train')

In [54]:
X_train = np.array(X_train, dtype = np.float32)
# X_train = rgb2gray(X_train)
X_train = X_train / 255
X_train, input_shape = fit_keras_channels(X_train)

print(X_train.shape)
print(input_shape)

(300004, 60, 160, 1)
(60, 160, 1)


In [55]:
Y_train = list(Y_train)
for i in range(len(Y_train)):
    Y_train[i] = text2vec(Y_train[i])
Y_train = np.asarray(Y_train)

print(Y_train.shape)

(300004, 248)


In [57]:
# 创建输入层
with tf.name_scope('inputs'):
    inputs = Input(shape=input_shape, name='inputs')

# 第一层卷积
with tf.name_scope('conv1'):
    conv1 = Conv2D(32, (3,3), name='conv1')(inputs)
    relu1 = Activation('relu', name='relu1')(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2), padding='same', name='pool1')(relu1)
    drop1 = Dropout(0.25)(pool1)

# 第二层卷积
with tf.name_scope('conv2'):
    conv2 = Conv2D(32, (3,3), name='conv2')(drop1)
    relu2 = Activation('relu', name='relu2')(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2), padding='same', name='pool2')(relu2)
    drop2 = Dropout(0.25)(pool2)

# 第三层卷积
with tf.name_scope('conv3'):
    conv3 = Conv2D(64, (3,3), name='conv3')(drop2)
    relu3 = Activation('relu', name='relu3')(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2), padding='same', name='pool3')(relu3)
    drop3 = Dropout(0.25)(pool3)

# 全连接层
with tf.name_scope('dense'):
    # 将池化后的数据摊平后输入全连接网络
    x = Flatten()(drop3)
    # Dropout
    x = Dropout(0.25)(x)
    # 创建4个全连接层,区分10类，分别识别4个字符
    x = [Dense(62, activation='softmax', name='func%d'%(i+1))(x) for i in range(4)]

# 输出层
with tf.name_scope('outputs'):
    # 将生成的4个字符拼接输出
    outs = Concatenate()(x)

In [58]:
model = Model(inputs=inputs, outputs=outs)
model.compile(optimizer='Adam', loss="binary_crossentropy", metrics=['accuracy'])

In [59]:
early_stopping = EarlyStopping(monitor = 'loss', patience = 2)

def scheduler(epoch):
    # 每隔10个epoch，学习率减小为原来的1/10
    if epoch % 10 == 0 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.1)
        print("lr changed to {}".format(lr * 0.1))
    return K.get_value(model.optimizer.lr)

reduce_lr = LearningRateScheduler(scheduler)

# from keras.callbacks import ModelCheckpoint
# checkpoint = ModelCheckpoint('save.h5', monitor='val_loss', save_weights_only=True,verbose=1,save_best_only=True, period=1)
history = model.fit(X_train, Y_train,
                   batch_size = 64,
                   epochs=100, verbose=1,
                   validation_split=0.2,
                   shuffle=True, callbacks=[early_stopping, reduce_lr])

Train on 240003 samples, validate on 60001 samples
Epoch 1/100
240003/240003 [==============================] - 83s 346us/step - loss: 0.0551 - acc: 0.9860 - val_loss: 0.0935 - val_acc: 0.9858
Epoch 2/100
240003/240003 [==============================] - 73s 304us/step - loss: 0.0347 - acc: 0.9892 - val_loss: 0.0871 - val_acc: 0.9865
Epoch 3/100
240003/240003 [==============================] - 73s 305us/step - loss: 0.0317 - acc: 0.9900 - val_loss: 0.0855 - val_acc: 0.9868
Epoch 4/100
240003/240003 [==============================] - 74s 308us/step - loss: 0.0301 - acc: 0.9904 - val_loss: 0.0856 - val_acc: 0.9868
Epoch 5/100
240003/240003 [==============================] - 74s 309us/step - loss: 0.0291 - acc: 0.9907 - val_loss: 0.0828 - val_acc: 0.9869
Epoch 6/100
240003/240003 [==============================] - 74s 310us/step - loss: 0.0283 - acc: 0.9909 - val_loss: 0.0833 - val_acc: 0.9870
Epoch 7/100
240003/240003 [==============================] - 74s 309us/step - loss: 0.0277 - acc:

In [70]:
model.save('captcha.h5')

In [89]:
X_test, Y_label = load_data('captcha_images')

In [90]:
Y_label

['007U',
 '00GJ',
 '00J5',
 '00KY',
 '00R6',
 '00V4',
 '00W3',
 '010B',
 '011Q',
 '016T',
 '01C6',
 '01M6',
 '01PY',
 '01W2',
 '0205',
 '0217',
 '025J',
 '0270',
 '02GD',
 '02HM',
 '02HN',
 '02LR',
 '02S8',
 '02US',
 '02V2',
 '02W3',
 '02XE',
 '03AV',
 '03F3',
 '03GY',
 '03N4',
 '03QY',
 '03WZ',
 '03X1',
 '04B9',
 '04HQ',
 '04PP',
 '04SQ',
 '04VU',
 '04WE',
 '04ZU',
 '05CZ',
 '05KA',
 '05UV',
 '05VL',
 '062E',
 '063G',
 '0677',
 '06D9',
 '06DQ',
 '06ET',
 '06FD',
 '06G8',
 '06JA',
 '06JG',
 '06LB',
 '06NM',
 '06QL',
 '06QR',
 '06TC',
 '06W3',
 '06ZS',
 '07CU',
 '07DC',
 '07E5',
 '07FH',
 '07H6',
 '07SQ',
 '07VC',
 '07X0',
 '07X4',
 '08NV',
 '08R1',
 '08U4',
 '099C',
 '09LP',
 '09MX',
 '09R9',
 '09S1',
 '09SD',
 '09YA',
 '09Z3',
 '0A8H',
 '0ACZ',
 '0ADK',
 '0AFP',
 '0AKD',
 '0ALB',
 '0AR5',
 '0AS1',
 '0B2F',
 '0B2Z',
 '0BC5',
 '0BLA',
 '0BMN',
 '0BTA',
 '0BVH',
 '0BZR',
 '0C3X',
 '0C49',
 '0C5A',
 '0C6Z',
 '0CFX',
 '0CMW',
 '0CR5',
 '0CVR',
 '0CW6',
 '0CZQ',
 '0D28',
 '0D8L',
 '0DH9',
 

In [91]:
X_test = np.array(X_test, dtype = np.float32)
# X_test = rgb2gray(X_test)
X_test = X_test / 255
X_test, input_shape = fit_keras_channels(X_test)

print(X_test.shape)
print(input_shape)

(9965, 60, 160, 1)
(60, 160, 1)


In [92]:
Y_test = list(Y_label)
for i in range(len(Y_test)):
    Y_test[i] = text2vec(Y_test[i])
Y_test = np.asarray(Y_test)

print(Y_test.shape)

(9965, 248)


In [93]:
pred = model.predict(X_test)

In [94]:
Y_pred = []
for i in range(len(Y_test)):
    Y_pred.append(vec2text(pred[i]))

In [96]:
count = 0
for i in range(len(Y_pred)):
    if Y_pred[i] == Y_label[i]:
        count += 1

In [97]:
count / len(Y_pred)

0.4401404917210236